# Pooling Layer

In [2]:
import tensorflow as tf
import pandas as pd

In [3]:
# 데이터를 준비하고
(input, target), _ = tf.keras.datasets.mnist.load_data()
input = input.reshape(60000, 28, 28, 1)     # 3차원으로 변환
target = pd.get_dummies(target)             # 원핫인코딩
print(input.shape, target.shape)

(60000, 28, 28, 1) (60000, 10)


In [4]:
# 모델을 만들고
X = tf.keras.layers.Input(shape=[28, 28, 1])
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)

H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 3)         78        
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 6)         456       
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 84)                201684    
                                                                 
 dense_1 (Dense)             (None, 10)                850       
                                                                 
Total params: 203,068
Trainable params: 203,068
Non-trainable

In [6]:
# 히든 레이어의 weights 파라미터 수 계산
84 * (2400 + 1) + 10 * (84 + 1) + 78 + 456

203068

## MaxPooling 적용

In [7]:
# 모델을 만들고(풀링 추가), CNN
X = tf.keras.layers.Input(shape=[28, 28, 1])

H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X)
H = tf.keras.layers.MaxPool2D()(H)                                      # 특징을 유지하기 위해 maxpooling을 사용한다.

H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(H)
H = tf.keras.layers.MaxPool2D()(H)

H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 3)         78        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 3)        0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 6)           456       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 6)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 96)                0   

In [9]:
model.fit(input, target, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.6025 - accuracy: 0.8751
Epoch 2/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1413 - accuracy: 0.9587
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1085 - accuracy: 0.9689
Epoch 4/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0941 - accuracy: 0.9732
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0834 - accuracy: 0.9763
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0813 - accuracy: 0.9761
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0761 - accuracy: 0.9782
Epoch 8/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0731 - accuracy: 0.9792
Epoch 9/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0724 - accuracy: 0.9796
Epoch 10/10
1875/1875 [==============================] - 11s 6ms/step - l

풀링을 쓰면 속도가 빨라지면서도 정확도는 떨어지지 않는다.

In [11]:
# 모델을 이용합니다. 
pred = model.predict(input[0:5])
pd.DataFrame(pred).round(2)

1/1 [==============================] - 0s 21ms/step


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.00,0.0,0.01,0.00,0.99,0.0,0.0,0.0,0.0
1,1.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
2,0.0,0.36,0.0,0.00,0.64,0.00,0.0,0.0,0.0,0.0
3,0.0,1.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,1.0


In [12]:
# 정답 확인
target[0:5]

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1


In [14]:
model.evaluate(input, target)

1875/1875 [==============================] - 7s 3ms/step - loss: 0.0688 - accuracy: 0.9804


[0.06882689893245697, 0.9804166555404663]